In [ ]:
#Createed on September 11, 2024, Data Preprocessing for Viral Bound Peptide Association with BK Viral infection
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
bk_data = pd.read_csv('/content/drive/MyDrive/ViralPeptidesv.BKVirus_CleanVersion.csv')
# bk_data.head()
# print(bk_data.shape)
# bk_data.isnull().sum()
bk_data_mod = bk_data.ffill()
bk_data_mod

In [ ]:
bk_data_mod.value_counts(['Transplant organ'])

In [ ]:
bk_data_mod['Disease'].replace('-','BKVAN',inplace = True)
bk_data_mod.value_counts('Disease')

In [ ]:
bk_data_mod.value_counts('Source')

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

le.fit(bk_data_mod['Transplant organ'])
numerical_organ = le.transform(bk_data_mod['Transplant organ'])
print(len(bk_data_mod['Transplant organ']))
print(bk_data_mod['Transplant organ'][0])
dict1 = {}
for i in range (0,225):
  dict1[bk_data_mod['Transplant organ'][i]] = numerical_organ[i]
bk_data_mod['Transplant organ'] = bk_data_mod['Transplant organ'].map(dict1).fillna(bk_data_mod['Transplant organ'])

In [ ]:
le.fit(bk_data_mod['Source'])
numerical_source = le.transform(bk_data_mod['Source'])
dict2 = {}
for i in range (0,225):
  dict2[bk_data_mod['Source'][i]] = numerical_source[i]
bk_data_mod['Source'] = bk_data_mod['Source'].map(dict2).fillna(bk_data_mod['Source'])

In [ ]:
le.fit(bk_data_mod['Disease'])
numerical_disease = le.transform(bk_data_mod['Disease'])
dict4 = {}
for i in range (0,225):
  dict4[bk_data_mod['Disease'][i]] = numerical_disease[i]
bk_data_mod['Disease'] = bk_data_mod['Disease'].map(dict4).fillna(bk_data_mod['Disease'])

In [ ]:
viralvals  = []
for i in range (0,225):
  temp = bk_data_mod['Viral load (copies/mL)'][i].split("×")
  temp[0].strip()
  newdigits = float(temp[0])
  temp[1].strip()
  ten = temp[1][0:3]
  expo = temp[1][3::]
  ten = int(ten)
  expo = int(expo)
  power = ten**expo
  viralload = newdigits * power
  viralvals.append(viralload)
dict3 = {}
for i in range (0,225):
  dict3[bk_data_mod['Viral load (copies/mL)'][i]] = viralvals[i]
bk_data_mod['Viral load (copies/mL)'] = bk_data_mod['Viral load (copies/mL)'].map(dict3).fillna(bk_data_mod['Viral load (copies/mL)'])

In [ ]:
del bk_data_mod['Sampling day']

In [ ]:
bk_data_mod['logvalue'] = np.log10(bk_data_mod['Viral load (copies/mL)'])


In [ ]:
plt.figure(figsize = (10,12))
plt.hist(bk_data_mod['logvalue'],edgecolor = 'black')
plt.show()

In [ ]:
plt.figure(figsize=(12,10))
sns.scatterplot(x=bk_data_mod['Number of polymorphisms'], y=bk_data_mod['logvalue'],hue= bk_data_mod['Source'])
plt.show()

In [ ]:
bk_data_mod.describe()

In [ ]:
bk_data_mod.head(225)

In [ ]:
y = bk_data_mod['logvalue']
X = bk_data_mod[['Number of polymorphisms'],['source']]

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2,random_state = 42)

In [ ]:
np.random.seed(42)

In [ ]:
import math
def truncate(number, decimals):
    factor = 10.0 ** decimals
    return math.trunc(number * factor) / factor

In [ ]:
reg = LinearRegression(random_state=42) #Linear Regression
reg.fit(X_train,y_train)
prediction = reg.predict(X_test)
print("Mean Absolute Error:     ",truncate(mean_absolute_error(y_test,prediction),6))
print("Mean Squared Error:      ",truncate(mean_squared_error(y_test,prediction),6))
print("Square Residual:         ",truncate(r2_score(y_test,prediction),6))
print("Root Mean Squared Error: ",str(truncate(mean_squared_error(y_test,prediction, squared = False),6))+'00')

In [ ]:
plt.figure(figsize=(8, 10))

# Plot real data
plt.plot(range(len(y_test)), y_test, label='True Values', color='blue',marker = 'o')

# Plot predicted data
plt.plot(range(len(prediction)), prediction, label='Predicted Data', color='red',marker='o')

# Add labels and legend
plt.xlabel('Number of Polymorphisms')
plt.ylabel('BK Viral Log Value')
plt.title('Raw Linear Regressor Predicted vs True')
plt.legend()

# Show the plot
plt.show()

In [ ]:
from sklearn import linear_model # Ridge Regression
reg1 = linear_model.Ridge(alpha = 0.5,random_state=42)
reg1.fit(X_train,y_train)
reg1_pred = reg1.predict(X_test)
print("Mean Absolute Error:    ",truncate(mean_absolute_error(y_test,reg1_pred),6))
print("Mean Squared Error:     ",str(truncate(mean_squared_error(y_test,reg1_pred),6))+'0')
print("Square Residual:        ",truncate(r2_score(y_test,reg1_pred),6))
print("Root Mean Square Error: ",truncate(mean_squared_error(y_test,reg1_pred,squared = False),6))

In [ ]:
plt.figure(figsize=(8, 10))

# Plot real data
plt.plot(range(len(y_test)), y_test, label='True Values', color='blue',marker = 'o')

# Plot predicted data
plt.plot(range(len(reg1_pred)), reg1_pred, label='Predicted Data', color='red',marker='o')

# Add labels and legend
plt.xlabel('Number of Polymorphisms')
plt.ylabel('BK Viral Log Value')
plt.title('Raw Ridge Regressor Predicted vs True')
plt.legend()

# Show the plot
plt.show()

In [ ]:
clf = linear_model.Lasso(alpha = 0.01) # Lasso : Note: 0.01 Alpha yields the lowest M.A. Error
clf.fit(X_train,y_train)
clf_pred = clf.predict(X_test)
print("Mean Absolute Error:    ",truncate(mean_absolute_error(y_test,clf_pred),6))
print("Mean Squared Error:     ",truncate(mean_squared_error(y_test,clf_pred),6))
print("Square Residual:        ",truncate(r2_score(y_test,clf_pred),6))
print("Root Mean Squared Error:",truncate(mean_squared_error(y_test,clf_pred, squared = False),6))
#Lasso : 1.067 Lowest Absolute Error

In [ ]:
plt.figure(figsize=(8, 10))

# Plot real data
plt.plot(range(len(y_test)), y_test, label='True Values', color='blue',marker = 'o')

# Plot predicted data
plt.plot(range(len(clf_pred)), clf_pred, label='Predicted Data', color='red',marker='o')

# Add labels and legend
plt.xlabel('Number of Polymorphisms')
plt.ylabel('BK Viral Log Value')
plt.title('Raw Lasso Regressor Predicted vs True')
plt.legend()

# Show the plot
plt.show()

In [ ]:
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
regr_svr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2),random_state=42)
regr_svr.fit(X_train,y_train)
regr_svr_pred = regr_svr.predict(X_test)
print("Mean Absolute Error:     ",truncate(mean_absolute_error(y_test,regr_svr_pred),6))
print("Mean Squared Error:      ",truncate(mean_squared_error(y_test,regr_svr_pred),6))
print("Square Residual:         ",truncate(r2_score(y_test,regr_svr_pred),6))
print("Root Mean Squared Error: ",truncate(mean_squared_error(y_test,regr_svr_pred,squared = False),6))

In [ ]:
plt.figure(figsize=(8, 10))

# Plot real data
plt.plot(range(len(y_test)), y_test, label='True Values', color='blue',marker = 'o')

# Plot predicted data
plt.plot(range(len(regr_svr_pred)), regr_svr_pred, label='Predicted Data', color='red',marker='o')

# Add labels and legend
plt.xlabel('Number of Polymorphisms')
plt.ylabel('BK Viral Log Value')
plt.title('Raw Support Vector Regressor Predicted vs True')
plt.legend()

# Show the plot

In [ ]:
print(np.log10(14))

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
regressor = DecisionTreeRegressor(random_state=42)
regressor.fit(X_train,y_train)
DTR_pred = regressor.predict(X_test)
print("Mean Absolute Error:    ",truncate(mean_absolute_error(y_test,DTR_pred),6))
print("Mean Squared Error:     ",truncate(mean_squared_error(y_test,DTR_pred),6))
print("Square Residual:        ",truncate(r2_score(y_test,DTR_pred),6))
print("Root Mean Squared Error:",truncate(mean_squared_error(y_test,DTR_pred, squared = False),6))

In [ ]:
plt.figure(figsize=(8, 10))

# Plot real data
plt.plot(range(len(y_test)), y_test, label='True Values', color='blue',marker='o')

# Plot predicted data
plt.plot(range(len(DTR_pred)), DTR_pred, label='Predicted Data', color='red',marker = 'o')

# Add labels and legend
plt.xlabel('Number of Polymorphisms')
plt.ylabel('BK Viral Log Value')
plt.title('Raw Decision Tree Regressor Predicted vs True')
plt.legend()

# Show the plot
plt.show()

In [ ]:
from sklearn.ensemble import RandomForestRegressor # Random Forest Regression
regr = RandomForestRegressor(max_depth = 2, random_state = 42)
regr.fit(X_train,y_train)
regr_prediction = regr.predict(X_test)
print("Mean Absolute Error:     ",truncate(mean_absolute_error(y_test,regr_prediction),6))
print("Mean Squared Error:      ",truncate(mean_squared_error(y_test,regr_prediction),6))
print("Square Residual:         ",truncate(r2_score(y_test,regr_prediction),6))
print("Root Mean Squared Error: ",truncate(mean_squared_error(y_test,regr_prediction, squared = False),6))

In [ ]:
# plt.figure(figsize=(8, 10))
# print(X_test.shape)
# print(y_test.shape)

# # Plot real data
plt.plot(range(len(y_test)), y_test, label='True Values', color='blue',marker='o')

# # Plot predicted data
plt.plot(range(len(regr_prediction)), regr_prediction, label='Predicted Data', color='red',marker='o')

# Add labels and legend
plt.xlabel('Index')
plt.ylabel('BK Viral Log Value')
plt.title('Raw Random Forest Regressor Predicted vs True')
plt.legend()

# Show the plot
plt.show()

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
distributions = {'max_depth':[2,3,4,5,6],'n_estimators':[100,125,150,175,200]}
rfg = RandomizedSearchCV(regr,distributions,random_state = 42,scoring = 'neg_mean_absolute_error')
search = rfg.fit(X_train,y_train)
search.best_params_
searchpath = pd.DataFrame(search.cv_results_)
print(searchpath)

In [ ]:
import scipy.interpolate as interp
print(distributions['max_depth'])
print(distributions['n_estimators'])
a = np.array(distributions['max_depth'])
b = np.array(distributions['n_estimators'])
listk = np.reshape(searchpath['mean_test_score'],(10,1)).T
# z_inter = interp.interp1d(np.arange(list1.size), list1)
# z_ = z_inter(np.linspace(0,list1.size-1,x.size))
a_inter = interp.interp1d(np.arange(a.size),a)
a_ = a_inter(np.linspace(0,a.size-1,listk.size))
b_inter = interp.interp1d(np.arange(b.size),b)
b_ = b_inter(np.linspace(0,b.size-1,listk.size))

fig = plt.figure(figsize = (25, 25))
ax = fig.add_subplot(111,projection = '3d')
ax.scatter(a_,b_,listk,linewidths = 4, alpha = 0.7, edgecolor = 'black',s = 700, c = listk)
ax.set_xlabel('Max Depth', fontsize=15,labelpad=20,weight='bold')
ax.set_ylabel('Number of Estimators', fontsize=15,labelpad=20,weight='bold')
ax.tick_params(axis='x', labelsize=15)
ax.tick_params(axis='y', labelsize=15)
ax.tick_params(axis='z', labelsize=15)

ax.set_zlabel('Negative Mean Absolute Error',fontsize=15,labelpad=20,weight='bold')
plt.title('Random Forest Randomized Search Results', fontsize=24,weight='bold')
plt.show()

In [ ]:
plt.plot(range(1,11),searchpath['mean_test_score'],marker = 'o',color='green')
plt.xlabel('Index')
plt.ylabel('Negative Mean Absolute Error')
plt.title('Randomized Search')

In [ ]:
plt.scatter(range(1,11),searchpath['mean_test_score'])

In [ ]:
from sklearn.model_selection import GridSearchCV
rfggsc = GridSearchCV(regr,distributions,scoring = 'neg_mean_absolute_error')
sgcs = rfggsc.fit(X_train,y_train,random_state=42)
# rfgpath = pd.DataFrame(rfggsc.cv_results)
# print(rfggsc.cv_results_)
path = pd.DataFrame(rfggsc.cv_results_)
print(path)
print(pd.concat([pd.DataFrame(rfggsc.cv_results_["params"]),pd.DataFrame(rfggsc.cv_results_["mean_test_score"], columns=["Negative Mean Absolute Error"])],axis=1))
# # plt.figure(figsize=(10,6))
# # plt.plot(range(1, 16), rfggsc.cv_results)


In [ ]:
print(sgcs.best_score_)

In [ ]:
print(sgcs.best_params_)
print(sgcs.best_score_)

In [ ]:
# plt.plot(range(1,26),path['mean_test_score'])
import scipy.interpolate as interp
print(distributions['max_depth'])
print(distributions['n_estimators'])
x = np.array(distributions['max_depth'])
y = np.array(distributions['n_estimators'])
list1 = np.reshape(path['mean_test_score'],(25,1)).T
# z_inter = interp.interp1d(np.arange(list1.size), list1)
# z_ = z_inter(np.linspace(0,list1.size-1,x.size))
x_inter = interp.interp1d(np.arange(x.size),x)
x_ = x_inter(np.linspace(0,x.size-1,list1.size))
y_inter = interp.interp1d(np.arange(y.size),y)
y_ = y_inter(np.linspace(0,y.size-1,list1.size))

fig = plt.figure(figsize = (25, 25))
ax = fig.add_subplot(111,projection = '3d')
ax.scatter3D(x_,y_,list1,linewidths = 3, alpha = 0.7, edgecolor = 'black',s = 500, c = list1)
ax.set_xlabel('Max Depth', fontsize=18,labelpad=20)
ax.set_ylabel('Number of Estimators', fontsize=18,labelpad=20)
ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
ax.tick_params(axis='z', labelsize=18)

ax.set_zlabel('Negative Mean Absolute Error',fontsize=18,labelpad=20)
plt.title('Random Forest Grid Search Results', fontsize=20)
plt.show()

In [ ]:
new_regr = RandomForestRegressor(max_depth=1,n_estimators=1000,random_state=42)
new_regr.fit(X_train,y_train)
new_regr_prediction = new_regr.predict(X_test)
print("Mean Absolute Error:    " ,truncate(mean_absolute_error(y_test,new_regr_prediction),6))
print("Mean Squared Error:     " ,truncate(mean_squared_error(y_test,new_regr_prediction),6))
print("Square Residual:        " ,truncate(r2_score(y_test,new_regr_prediction),6))
print("Root Mean Squared Error:",truncate(mean_squared_error(y_test,new_regr_prediction, squared = False),6))

In [ ]:
import joblib
joblib.dump(new_regr, '/content/drive/MyDrive/model.joblib')

In [ ]:
plt.figure(figsize=(8, 10))

# Plot real data
plt.plot(range(len(y_test)), y_test, label='True Values', color='blue',marker='o')

# Plot predicted data
plt.plot(range(len(new_regr_prediction)), new_regr_prediction, label='Predicted Data', color='red',marker = 'o')

# Add labels and legend
plt.xlabel('Number of Polymorphisms')
plt.ylabel('BK Viral Log Value')
plt.title('Tuned Random Forest Regressor Predicted vs True')
plt.legend()

# Show the plot
plt.show()